#### Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
##### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [2]:
keywords = ['python', 'парсинг']
#keywords = ['Netology']
url = 'https://habr.com/ru/all'
req = requests.get(url)
req

<Response [200]>

In [3]:
soup = BeautifulSoup(req.text, 'html.parser')
links = soup.find_all('a', class_='post__title_link')
habr_news = pd.DataFrame()
for link in links:
    href = link.attrs.get('href')
    response = requests.get(href)
    time.sleep(0.3)
    article = BeautifulSoup(response.text, 'html.parser')
    post = article.find('div', id = 'post-content-body').get_text()
    for word in keywords:
        a = re.search(word, post, flags=re.IGNORECASE)
        if a is not None:
            date = pd.to_datetime(article.find('span', class_ = 'post__time').get('data-time_published')).date()
            title = article.find('span', class_ = 'post__title-text').text
            row = {'date': date, 'title': title, 'link': href}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
            #print(row)
            break
if len(habr_news) > 0:
    display(habr_news.reset_index())
else:
    print(f'Ключевые слова {keywords} не найдены')

,index,date,title,link
0,0,2021-05-22,Пройти до конца: почему бросать и снова начина...,https://habr.com/ru/company/skillfactory/blog/...
1,0,2021-05-22,Подборка 150+ ресурсов для управления и работы...,https://habr.com/ru/post/558770/
2,0,2021-05-22,Детектор космических лучей. Часть 3: завершени...,https://habr.com/ru/company/ruvds/blog/557594/


Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [4]:
soup = BeautifulSoup(req.text, 'html.parser')
links = soup.find_all('a', class_='post__title_link')
habr_news = pd.DataFrame()
for link in links:
    href = link.attrs.get('href')
    response = requests.get(href)
    time.sleep(0.3)
    article = BeautifulSoup(response.text, 'html.parser')
    post = article.find('div', id = 'post-content-body').get_text()
    for word in keywords:
        a = re.search(word, post, flags=re.IGNORECASE)
        if a is not None:
            date = pd.to_datetime(article.find('span', class_ = 'post__time').get('data-time_published')).date()
            title = article.find('span', class_ = 'post__title-text').text
            content = article.find('div', class_ = 'post__body').text
            row = {'date': date, 'title': title, 'link': href, 'content': content}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
            #print(row)
            break
if len(habr_news) > 0:
    display(habr_news.reset_index())
else:
    print(f'Ключевые слова {keywords} не найдены')

,index,date,title,link,content
0,0,2021-05-22,Пройти до конца: почему бросать и снова начина...,https://habr.com/ru/company/skillfactory/blog/...,\nМногие из нас начинают и бросают много самых...
1,0,2021-05-22,Подборка 150+ ресурсов для управления и работы...,https://habr.com/ru/post/558770/,\nПривет! На связи компания KODE. Мы занимаемс...
2,0,2021-05-22,Детектор космических лучей. Часть 3: завершени...,https://habr.com/ru/company/ruvds/blog/557594/,"\n\r\nВ первой части серии мы разобрали, что т..."


##### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]
В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [13]:
import json

url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"

payload = {"emailAddresses":["fff@mail.ru"]}

# Поиск того, какие ключи надо подставить, чтобы определить, где находится информация о взломах эл. адреса, самостоятельно
# не сделала. На лекции такого не объясняли. Обращалась за помощью к своему источнику. Без помощи бы не справилась. 
# Принципы поиска внутри html-кода остались неусвоенными, очень большая структура кода, кроме заголовков, текста, 
# простой разметки, ссылок, остальной код остается непонятным.
headers = {
    'Content-Type': "application/json",
    'Vaar-Version': "0",
    'Vaar-Header-App-Product': "hackcheck-web-avast",
    'Vaar-Header-App-Product-Name': "hackcheck-web-avast",
    'Vaar-Header-App-Build-Version': "1.0.0",
    }

response = requests.post(url, data=json.dumps(payload), headers=headers)
breaches = json.loads(response.text)
breaches
# Не всегда срабатывает, т.к. сайт https://www.avast.com/hackcheck регулярно запрашивает про "Я не робот".

{'breaches': {'25953': {'breachId': 25953,
   'site': 'ta-bao.com',
   'recordsCount': 240992,
   'description': 'In November 2020, a collection of over 23,000 breached sites was leaked on several hacking forums and Telegram channels. These breached sites originated from Cit0Day, a now-defunct private subscription service marketed towards criminals. The leaked data primarily includes email addresses and passwords that Cit0Day offered for a daily or monthly subscription fee.',
   'publishDate': '2020-12-03T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 239943, 'emails': 240992}},
  '16372': {'breachId': 16372,
   'site': 'bookmate.com',
   'recordsCount': 7775238,
   'description': 'In July 2018, Bookmate was allegedly breached. The stolen data contains usernames, passwords, salts, email addresses and additional personal information. This data has been sold on at least one dark web market and is also being privately shared among several criminal networks.\r\n',
   'publishDa

In [14]:
print(breaches)

{'breaches': {'25953': {'breachId': 25953, 'site': 'ta-bao.com', 'recordsCount': 240992, 'description': 'In November 2020, a collection of over 23,000 breached sites was leaked on several hacking forums and Telegram channels. These breached sites originated from Cit0Day, a now-defunct private subscription service marketed towards criminals. The leaked data primarily includes email addresses and passwords that Cit0Day offered for a daily or monthly subscription fee.', 'publishDate': '2020-12-03T00:00:00Z', 'statistics': {'usernames': 0, 'passwords': 239943, 'emails': 240992}}, '16372': {'breachId': 16372, 'site': 'bookmate.com', 'recordsCount': 7775238, 'description': 'In July 2018, Bookmate was allegedly breached. The stolen data contains usernames, passwords, salts, email addresses and additional personal information. This data has been sold on at least one dark web market and is also being privately shared among several criminal networks.\r\n', 'publishDate': '2019-03-20T00:00:00Z', 

In [17]:
breaches['summary']
leaks = pd.DataFrame()
for mail in breaches['summary']:
#     print(mail)
    for breach in breaches['summary'][mail]["breaches"]:
#         print(breach)
#         print(breaches["breaches"][str(breach)]['description'])
        publish_date = breaches["breaches"][str(breach)]['publishDate']
        site = breaches["breaches"][str(breach)]['site']
        description = breaches["breaches"][str(breach)]['description']
        row = {'publish_date': publish_date, 'site': site, 'description': description}
        leaks = pd.concat([leaks, pd.DataFrame([row])])
leaks

,publish_date,site,description
0,2020-12-03T00:00:00Z,ta-bao.com,"In November 2020, a collection of over 23,000 ..."
0,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,2020-05-28T00:00:00Z,stalker.so,"In January 2020, the Russian multiplayer video..."
0,2019-10-10T00:00:00Z,ozonemc.ru,"At an unconfirmed date, two Russian Minecraft ..."
0,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,2019-05-09T00:00:00Z,lookbook.nu,"In October 2016, fashion blog Lookbook's datab..."
0,2020-12-10T00:00:00Z,pfv.gostika.ru,"In November 2020, a collection of over 23,000 ..."
0,2019-08-08T00:00:00Z,borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
0,2020-04-02T00:00:00Z,romwe.com,"At an unconfirmed date, the online fashion ret..."
0,2020-12-17T00:00:00Z,btravel.ru,"In November 2020, a collection of over 23,000 ..."


Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>